<a href="https://colab.research.google.com/github/rtgrockzz/machine_learning_examples/blob/master/mf_model_v1(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [0]:
from fastai import *
from fastai.data_block import *
from torch.utils.data import DataLoader, Dataset
from torch import *
from fastai.tabular import *
import pandas as pd
import numpy as np
from torchsummary import summary
#cuda.init()
#cuda=torch.device('cuda')

In [0]:
print(torch.cuda.is_available())
dev = torch.device('cuda') 

True


In [0]:
from torch.utils.data import Dataset, DataLoader

class TabularDataset(Dataset):
    def __init__(self, data, cat_cols=None, output_col=None):
        self.n = data.shape[0]
        if output_col:
            self.y = data[output_col].astype(np.float32).values
        else:
            self.y =  np.zeros((self.n, 1))

        self.cat_cols = cat_cols if cat_cols else []
        self.cont_cols = [col for col in data.columns
                          if col not in self.cat_cols + [output_col]]

        if self.cont_cols:
            print(data[self.cont_cols].shape)
            self.cont_X = data[self.cont_cols].astype(np.float32).values.reshape(data[self.cont_cols].shape[0],1, data[self.cont_cols].shape[1])
            print(self.cont_X.shape)
        else:
            self.cont_X = np.zeros((self.n, 1))

        if self.cat_cols:
            self.cat_X = data[cat_cols].astype(np.float32).values
        else:
            self.cat_X =  np.zeros((self.n, 1))

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        return self.cont_X[idx], self.y[idx]
    
    def is_empty(self):
        return False


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls

drive  pred_on_train.csv  sample_data


In [0]:
df=pd.read_csv('drive/My Drive/training_dataset.csv')
#df=df.iloc[:,4:-1]*100
#df.head()


In [0]:
df.shape

(3333, 789)

In [0]:
output_feature = "Y"

In [0]:
df['mf_id_rem']=df['mf_id']%10

In [0]:
df_train=df[~((df['mf_id_rem']==1) | (df['mf_id_rem']==2) | (df['mf_id_rem']==6))]

In [0]:
df_valid=df[((df['mf_id_rem']==1) | (df['mf_id_rem']==2) | (df['mf_id_rem']==6))]

In [0]:
df_valid.shape

(667, 790)

In [0]:
#df_train

In [0]:
df_train.shape

(2666, 790)

In [0]:
df_train=df_train.drop(['mf_id_rem'], axis=1)
df_valid=df_valid.drop(['mf_id_rem'], axis=1)
df=df.drop(['mf_id_rem'], axis=1)

In [0]:
df_train.shape, df_valid.shape

((2666, 789), (667, 789))

In [0]:
y_train=df_train['Y']*100

In [0]:
df_train_copy_1=df_train.iloc[:,:-1]

df_train=df_train.iloc[:,4:-1]*100
#df_train

In [0]:
#y_train.describe()

In [0]:
y_valid=df_valid['Y']*100

In [0]:
#y_valid.describe()

In [0]:
df_valid_copy_1=df_valid.iloc[:,:-1]

df_valid=df_valid.iloc[:,4:-1]*100


In [0]:
#df_valid_copy_1

In [0]:
#df_valid

In [0]:
df_valid_copy= pd.merge(df_valid_copy_1,df_valid)

In [0]:
df_train.shape,df_valid.shape

((2666, 784), (667, 784))

In [0]:
#df_valid_copy_1

In [0]:
2866/3333

0.8598859885988599

In [0]:
#df_train.sum()

In [0]:
print(df_train.shape),
df_train = df_train.astype(np.float32).values.reshape(df_train.shape[0],1, df_train.shape[1])
df_train=  torch.Tensor(df_train)
df_valid = df_valid.astype(np.float32).values.reshape(df_valid.shape[0],1, df_valid.shape[1])
df_valid=  torch.Tensor(df_valid)

y_train=y_train.astype(np.float32).values
y_valid=y_valid.astype(np.float32).values

y_train=  torch.Tensor(y_train)
y_valid= torch.Tensor(y_valid)


df_train.shape,df_valid.shape

(2666, 784)


(torch.Size([2666, 1, 784]), torch.Size([667, 1, 784]))

In [0]:
df_train.shape,df_valid.shape
#df_train.cuda()
#df_valid.cuda()
#y_train.cuda()
#y_valid.cuda()

(torch.Size([2666, 1, 784]), torch.Size([667, 1, 784]))

In [0]:
#train_ds = TabularDataset(data=df_train,output_col=output_feature)
#valid_ds = TabularDataset(data=df_valid,output_col=output_feature)

#datas=TabularDataset(data=df,                              output_col=output_feature)
#datas

In [0]:
#torch.device=cuda
train_ds=TensorDataset(df_train,y_train)
#print(train_ds.device)
valid_ds=TensorDataset(df_valid,y_valid)
#train_ds.cuda()
#train_ds=train_ds.to('cuda')
#print(train_ds.device)

In [0]:
TensorDataset??


In [0]:
#data = DataBunch.create(train_ds, valid_ds, bs=64, num_workers=1)

In [0]:
# x,y=data.one_batch()
# x[torch.isnan(x)] = 0
# x.shape,y.shape

In [0]:


def preprocess(x):
    size=x.shape[0]
    x=x.view(size)
    x[torch.isnan(x)] = 0
    return x


In [0]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

models=nn.Sequential(
    nn.BatchNorm1d(1),
    nn.Conv1d(1,256,kernel_size=5,bias=True),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=5),
    nn.Conv1d(256,512,kernel_size=5,bias=True),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=5),
    nn.Conv1d(512,1024,kernel_size=5,bias=True),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    #nn.MaxPool1d(kernel_size=5),
    #nn.Conv1d(256,512,kernel_size=5,bias=True),
    #nn.BatchNorm1d(512),
    #nn.ReLU(),
    #nn.MaxPool1d(kernel_size=5),
    #nn.Conv1d(512,1024,kernel_size=5,bias=True),
#     nn.BatchNorm1d(1024),
#     nn.ReLU(),
     nn.AdaptiveMaxPool1d(1),#concat avg
     Lambda(lambda x: x.view(x.size(0),-1)),
     nn.Linear(1024,512),
     nn.BatchNorm1d(512),
     nn.ReLU(),
     nn.Dropout(),
     nn.Linear(512,256),
     nn.ReLU(),
     nn.BatchNorm1d(256),
     nn.Dropout(),
     nn.Linear(256,1)
    
    
)

#learn = Learner(data, models, loss_func =  torch.nn.functional.l1_loss ,metrics=[torch.nn.functional.mse_loss])

Sequential(
  (0): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Conv1d(1, 256, kernel_size=(5,), stride=(1,))
  (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ReLU()
  (4): MaxPool1d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (5): Conv1d(256, 512, kernel_size=(5,), stride=(1,))
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): MaxPool1d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (9): Conv1d(512, 1024, kernel_size=(5,), stride=(1,))
  (10): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): AdaptiveMaxPool1d(output_size=1)
  (13): Lambda()
  (14): Linear(in_features=1024, out_features=512, bias=True)
  (15): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): ReLU()
  (17): Dropout(p=0.5)
  (18): Linear(in_

In [0]:
models2=nn.Sequential(
   nn.BatchNorm1d(1),
   nn.Conv1d(1,256,kernel_size=5,bias=True),
   nn.BatchNorm1d(256),
   nn.ReLU(),
   nn.MaxPool1d(kernel_size=5),
   # nn.Dropout(),
   nn.Conv1d(256,512,kernel_size=5,bias=True),
   nn.BatchNorm1d(512),
   nn.ReLU(),
   nn.MaxPool1d(kernel_size=5),
    #nn.Dropout(),
   nn.Conv1d(512,1024,kernel_size=5,bias=True),
   nn.BatchNorm1d(1024),
   nn.ReLU(),
    #nn.Dropout(),
   nn.AdaptiveMaxPool1d(1),#concat avg
    Lambda(lambda x: x.view(x.size(0),-1)),
    nn.Linear(1024,512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(512,256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(),
    #nn.Linear(256,1),
    
    nn.Linear(256,128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Linear(64,1))


In [0]:
#for p in models.parameters():
#    p.data.fill_(0)

loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(models.parameters(), lr=1e-5)
bs=64
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs ),
    )
def loss_batch(models, loss_func, xb, yb, opt=None):
    xb.to(dev)
    yb.to(dev)
    y_pred=models(xb)
    y_pred=preprocess(y_pred)
    loss = loss_func(y_pred, yb)
   # print('pred: ',y_pred,'y :',yb,'loss',loss)
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step() 
    return loss.item(), len(xb)
  
def fit(epochs, models, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):


        models.cuda()
        models=models.train()
        for xb, yb in train_dl:
          x=xb.to(dev)
          y=yb.to(dev)
          bl,s=loss_batch(models.to(dev), loss_func, x, y, opt)
            

        models=models.eval()
        with torch.no_grad():
            losses=[]
            nums=[]
            for xb, yb in valid_dl:
              x=xb.to(dev)
              y=yb.to(dev)
              losses_1, nums_1 = loss_batch(models, loss_func, x, y)
              losses.append(losses_1)
              nums.append(nums_1)
              

            #losses, nums = zip(*[loss_batch(models, loss_func, xb, yb) for xb, yb in valid_dl])
            
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print('loss:',epoch,bl,val_loss )


In [0]:
train_dl,valid_dl=get_data(train_ds, valid_ds, bs)
models2.to(dev)
optimizer = torch.optim.Adam(models2.parameters(), lr=1e-3)
fit(5, models2, loss_func, optimizer, train_dl, valid_dl)

optimizer = torch.optim.Adam(models2.parameters(), lr=1e-4)
fit(8, models2, loss_func, optimizer, train_dl, valid_dl)

optimizer = torch.optim.Adam(models2.parameters(), lr=1e-5)
fit(10, models2, loss_func, optimizer, train_dl, valid_dl)



loss: 0 37.978851318359375 22.97395526594308
loss: 1 27.113468170166016 16.90135197446443
loss: 2 49.560977935791016 18.254406756010727
loss: 3 12.46082592010498 16.00760947460535
loss: 4 7.5730671882629395 22.857353370586434
loss: 0 6.652295112609863 12.081309724604708
loss: 1 6.955573558807373 9.447634260872494
loss: 2 6.950926303863525 8.128837193684957
loss: 3 4.241746425628662 7.737910360768102
loss: 4 6.738016605377197 8.562889157742754
loss: 5 3.506566286087036 7.387579307384577
loss: 6 4.123189449310303 8.31851224884994
loss: 7 7.695555686950684 8.867319260043898
loss: 0 3.033552885055542 7.90200185704267
loss: 1 3.7090861797332764 8.21659913556329
loss: 2 9.39881420135498 8.276899925414947
loss: 3 4.42564582824707 8.052547030184401
loss: 4 5.567952632904053 8.483943187135985
loss: 5 3.3391833305358887 7.811338496172446
loss: 6 7.397759437561035 7.51161619498097
loss: 7 5.202752113342285 7.470788941390511
loss: 8 5.13186502456665 7.535996678708376
loss: 9 5.239177703857422 7.64

In [0]:
y_pred=models(df_valid)
print(df_valid_copy_1.shape)
df_valid_copy_1['y_real']=y_valid
df_valid_copy_1['y_pred']=y_pred.view(y_pred.shape[0])
#res_df['y_pred']=y_pred.view(y_pred.shape[0])


NameError: ignored

In [0]:
y_pred=models(df_valid)

In [0]:
df_valid_copy_1.to_csv('pred_on_valid.csv')

In [0]:
train_y_pred=models(df_train)
print(train_y_pred.shape)
df_train_copy_1['y_real']=y_train
df_train_copy_1['y_pred']=train_y_pred.view(train_y_pred.shape[0])
#res_df['y_pred']=y_pred.view(y_pred.shape[0])


NameError: ignored

In [0]:
df_train_copy_1.to_csv('pred_on_train.csv')

In [0]:


import torch.nn.functional as F
import torch.nn as nn


class Net_Resnet(nn.Module):
    def __init__(self):
        super(Net_Resnet,self).__init__()
        self.fc1=nn.Conv1d(1,256,kernel_size=5,bias=True)
        self.bn1=nn.BatchNorm1d(256)
        #self.r=nn.ReLU()
        self.mp1=nn.MaxPool1d(kernel_size=5)
        self.conv2=nn.Conv1d(256,512,kernel_size=5,bias=True)
        self.bn2=nn.BatchNorm1d(512)
        self.conv3=nn.Conv1d(512,1024,kernel_size=5,bias=True)
        self.bn3=nn.BatchNorm1d(1024)
        self.lin1=nn.Linear(1024,512)
        self.bn4=nn.BatchNorm1d(512)
        self.lin2=nn.Linear(512,256)
        self.bn5=nn.BatchNorm1d(256)
        self.dp=nn.Dropout(),
        self.lin3=nn.Linear(256,1)
        self.am=nn.AdaptiveAvgPool1d(1)
        self.ap=nn.AdaptiveMaxPool1d(1)
        
    def forward(self,x):
        x=nn.BatchNorm1d(1)(x)
        op=self.fc1(x)
        op=self.bn1(op)
        op=F.relu(op)
        op=self.mp1(op)+op
        op2=self.conv2(op)
        op2=self.bn2(op)
        op2=F.relu(op)
        op2=self.mp1(op)+op
        op3=self.conv3(op)
        op3=self.bn3(op)
        op3=F.relu(op)
        op4=self.am(op1)
        #op5=self.ap(op1)
        #op4=torch.cat([op4,op5],dim=1)
        op4=op4.view(op4.size(0),-1)
        op4=self.lin1(op4)
        op4=self.bn4(op4)
        op4=F.relu(op4)+op4
        op4=F.dropout(op4)
        op4=self.lin2(op4)
        op4=F.relu(op4)
        op4=self.bn5(op4)
        op4=F.dropout(op4)
        op4=self.lin3(op4)
        return op4
        
        
        
        
        
        

__main__.Net_Resnet

In [0]:
models3=nn.Sequential(
   nn.BatchNorm1d(1),
   nn.Conv1d(1,256,kernel_size=5,bias=True),
   nn.BatchNorm1d(256),
   nn.ReLU(),
   nn.MaxPool1d(kernel_size=5),
    nn.Dropout(.2),
   nn.Conv1d(256,512,kernel_size=5,bias=True),
   nn.BatchNorm1d(512),
   nn.ReLU(),
   nn.MaxPool1d(kernel_size=5),
    nn.Dropout(.2),
   nn.Conv1d(512,1024,kernel_size=5,bias=True),
   nn.BatchNorm1d(1024),
   nn.ReLU(),
    nn.Dropout(.2),
   nn.AdaptiveMaxPool1d(1),#concat avg
    Lambda(lambda x: x.view(x.size(0),-1)),
    nn.Linear(1024,512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.Linear(512,256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(),
    #nn.Linear(256,1),
    
    nn.Linear(256,128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Linear(64,1))

In [0]:
# loss_func = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(models.parameters(), lr=1e-5)
bs=64
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs ),
    )
def loss_batch(models, loss_func, xb, yb, opt=None):
    xb.to(dev)
    yb.to(dev)
    y_pred=models(xb)
    y_pred=preprocess(y_pred)
    loss = loss_func(y_pred, yb)
   # print('pred: ',y_pred,'y :',yb,'loss',loss)
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step() 
    return loss.item(), len(xb)
  
def fit(epochs, models, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):


        models.cuda()
        models=models.train()
        for xb, yb in train_dl:
          x=xb.to(dev)
          y=yb.to(dev)
          bl,s=loss_batch(models.to(dev), loss_func, x, y, opt)
            

        models=models.eval()
        with torch.no_grad():
            losses=[]
            nums=[]
            for xb, yb in valid_dl:
              x=xb.to(dev)
              y=yb.to(dev)
              losses_1, nums_1 = loss_batch(models, loss_func, x, y)
              losses.append(losses_1)
              nums.append(nums_1)
              

            #losses, nums = zip(*[loss_batch(models, loss_func, xb, yb) for xb, yb in valid_dl])
            
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print('loss:',epoch,bl,val_loss )


In [0]:
train_dl,valid_dl=get_data(train_ds, valid_ds, bs)
models3.to(dev)
optimizer = torch.optim.Adam(models3.parameters(), lr=1e-3)
fit(5, models3, loss_func, optimizer, train_dl, valid_dl)

optimizer = torch.optim.Adam(models3.parameters(), lr=1e-4)
fit(8, models3, loss_func, optimizer, train_dl, valid_dl)

optimizer = torch.optim.Adam(models3.parameters(), lr=1e-5)
fit(10, models3, loss_func, optimizer, train_dl, valid_dl)


loss: 0 51.72972869873047 21.932305214465828
loss: 1 31.602737426757812 17.554887211126186
loss: 2 22.581558227539062 26.790912854081686
loss: 3 17.485519409179688 11.521360666140623
loss: 4 21.417343139648438 20.160024119638788
loss: 0 13.039652824401855 19.67725239629331
loss: 1 18.034496307373047 16.380012071829686
loss: 2 10.613177299499512 11.646270689042076
loss: 3 11.092193603515625 17.855353445484898
loss: 4 7.487906455993652 16.703441511208506
loss: 5 6.933390140533447 14.21646446159397
loss: 6 6.7008843421936035 21.05094574523651
loss: 7 6.883625030517578 13.604678601518028
loss: 0 9.85671615600586 12.184458316534176
loss: 1 8.166848182678223 13.436518152971853
loss: 2 7.734708309173584 11.994268631827884
loss: 3 7.905764579772949 14.273823550794793
loss: 4 8.26514720916748 12.507326086064328
loss: 5 7.973119258880615 11.954154084647435
loss: 6 11.217812538146973 12.431790481979164
loss: 7 11.261299133300781 13.242792026571248
loss: 8 8.213129043579102 12.57228602533755
loss:

In [0]:
models4=nn.Sequential(
   nn.BatchNorm1d(1),
   nn.Conv1d(1,256,kernel_size=5,bias=True),
   nn.BatchNorm1d(256),
   nn.ReLU(),
   nn.MaxPool1d(kernel_size=5),
    nn.Dropout(.3),
   nn.Conv1d(256,512,kernel_size=5,bias=True),
   nn.BatchNorm1d(512),
   nn.ReLU(),
   nn.MaxPool1d(kernel_size=5),
    nn.Dropout(.3),
   nn.Conv1d(512,1024,kernel_size=5,bias=True),
   nn.BatchNorm1d(1024),
   nn.ReLU(),
    nn.Dropout(.3),
   nn.AdaptiveMaxPool1d(1),#concat avg
    Lambda(lambda x: x.view(x.size(0),-1)),
    nn.Linear(1024,512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(.3),
    nn.Linear(512,256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(),
    #nn.Linear(256,1),
    
    nn.Linear(256,128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Linear(64,1))

In [0]:
# loss_func = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(models.parameters(), lr=1e-5)
bs=64
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs ),
    )
def loss_batch(models, loss_func, xb, yb, opt=None):
    xb.to(dev)
    yb.to(dev)
    y_pred=models(xb)
    y_pred=preprocess(y_pred)
    loss = loss_func(y_pred, yb)
   # print('pred: ',y_pred,'y :',yb,'loss',loss)
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step() 
    return loss.item(), len(xb)
  
def fit(epochs, models, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):


        models.cuda()
        models=models.train()
        for xb, yb in train_dl:
          x=xb.to(dev)
          y=yb.to(dev)
          bl,s=loss_batch(models.to(dev), loss_func, x, y, opt)
            

        models=models.eval()
        with torch.no_grad():
            losses=[]
            nums=[]
            for xb, yb in valid_dl:
              x=xb.to(dev)
              y=yb.to(dev)
              losses_1, nums_1 = loss_batch(models, loss_func, x, y)
              losses.append(losses_1)
              nums.append(nums_1)
              

            #losses, nums = zip(*[loss_batch(models, loss_func, xb, yb) for xb, yb in valid_dl])
            
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print('loss:',epoch,bl,val_loss )


In [0]:
train_dl,valid_dl=get_data(train_ds, valid_ds, bs)
models4.to(dev)
optimizer = torch.optim.Adam(models4.parameters(), lr=1e-3)
fit(5, models4, loss_func, optimizer, train_dl, valid_dl)

optimizer = torch.optim.Adam(models4.parameters(), lr=1e-4)
fit(8, models4, loss_func, optimizer, train_dl, valid_dl)

optimizer = torch.optim.Adam(models4.parameters(), lr=1e-5)
fit(10, models4, loss_func, optimizer, train_dl, valid_dl)


loss: 0 48.60853958129883 22.16751938209362
loss: 1 40.42605972290039 17.91625677163097
loss: 2 23.86737060546875 19.291614403789012
loss: 3 20.722795486450195 14.585944923503824
loss: 4 20.219369888305664 15.08637025509996
loss: 0 15.893720626831055 12.914915396534521
loss: 1 11.72498607635498 11.651954845331241
loss: 2 13.951017379760742 12.007763661008546
loss: 3 20.102005004882812 11.91882430309656
loss: 4 21.881540298461914 19.35405328141517
loss: 5 11.719096183776855 13.774480114812436
loss: 6 18.678892135620117 14.103989701221014
loss: 7 11.821447372436523 11.783661703655923
loss: 0 11.843772888183594 16.76148133656789
loss: 1 12.706628799438477 15.777908173636876
loss: 2 11.854844093322754 16.098079180967684
loss: 3 12.636103630065918 14.44411037708151
loss: 4 13.957880020141602 14.401370065680508
loss: 5 7.6250834465026855 15.814554955112166
loss: 6 13.022941589355469 14.132905918618906
loss: 7 12.526524543762207 15.280075812447018
loss: 8 12.570005416870117 14.758630919849676

In [0]:
print(c)

In [0]:
for i in c :
  print(i)

(1, 5)
(2, 5)
(3, 5)
(4, 5)
(5, 5)
